In [1]:
import requests
import json
import math
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from qgridnext import show_grid
from datetime import datetime, timedelta
import sys  
import os
import pandas_ta as ta


# Ensure the directories are in the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Data_Management')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Universe_Selection')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Signal_Generation')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Risk_Management')))

# Import the modules
from data import Data
from calculations import Calculations
from coarse import Coarse_1 as Coarse
from fine import Fine_1 as Fine
from entry_signal import Trend_Following, Mean_Reversion
from tail_risk import Stop_Loss, Take_Profit

                                close  creturns      high  log_return  \
date                coin                                                
2020-12-31 19:00:00 BTCUSDT  29331.69       NaN  29600.00         NaN   
                    ETHUSDT    728.91       NaN    749.00         NaN   
2021-01-01 19:00:00 BTCUSDT  32178.33  1.097050  33300.00    0.092625   
                    ETHUSDT    774.56  1.062628    787.69    0.060745   
2021-01-02 19:00:00 BTCUSDT  33000.05  1.125065  34778.11    0.025216   
                    ETHUSDT    978.28  1.342114   1011.07    0.233501   
2021-01-03 19:00:00 BTCUSDT  31988.71  1.090585  33600.00   -0.031126   
                    ETHUSDT   1041.43  1.428750   1162.97    0.062554   
2021-01-04 19:00:00 BTCUSDT  33949.53  1.157435  34360.00    0.059492   
                    ETHUSDT   1099.56  1.508499   1134.60    0.054315   
2021-01-05 19:00:00 BTCUSDT  36769.36  1.253571  36939.21    0.079790   
                    ETHUSDT   1208.42  1.657845   1

In [2]:
symbols = ['BTCUSDT', 'ETHUSDT']
start_time = dt.datetime(2024, 1, 1)
end_time = dt.datetime(2024, 2, 1)
timeframe = '1h'
data = Data(symbols, timeframe, start_time, end_time).df
#Perform coarse analysis and filtering
coarse = Coarse()
fine = Fine()
tf = Trend_Following()
mr = Mean_Reversion()
cal = Calculations()

df = coarse.volume_flag(data, 1000)
df = coarse.sort_by_volume(df)
df = coarse.sort_by_std(df, 2, 2)
df = fine.above_ema(df, 2)
#Generate a signal (just for the sake of applying the risk management)
str_length = 10
str_mult = 3
_df = tf.supertrend_signals(df, str_length, str_mult)

#Perform some calculations
_df = cal.trades(_df)
_df = cal.strategy_returns(_df)
_df = cal.sessions(_df)
_df

AttributeError: 'numpy.ndarray' object has no attribute 'shift'

## Position

To make a position columns, we have to take into account overlapping positions that might yield better risk adjusted performance if the state that we are trying to exploit continues. Example, if we are buying in a trend, and we get another signal while price pullbacks, and we don't get stopped from the first signal. If the second signal is strong enough or assuming that trend following will continue, we are better to keep holding the first signal which would not only increase performance, but this would also lower the trading costs (depeninding on the broker)

Note that we need to make the positions columns then it we would translate it to sessions (this would make the code reusable). Position column is formed by the trading signal column (should be in the calculations module)
What we want is a trade signal to be triggered then:

-> If we won't take any new trades until the current trade is closed (Basically, stay in the trade and ignore the 	new trade): 
1. Position column = ffill() trading signal column (we want to create a position whenever we are in a trade) 
2. Calculate trades, strategy returns, sessions (as usual) (note that at first, sessions would = to position)
2. Loop on the len(df):\
	Apply SL and TP\
	if trading signal != position column (trading signal == 1 and position column == 0): \
		position column = trading signal\
		position column.ffill()\
=> Note that sl and take profit would loop on every session which does not affect this process as sessions are updated on each loop which is what we need as tp and sl is applied on sessions not positions

-> If we want to take an additional trade while keeping the current open trade: 
1. Position column = cumsum() the trading signals (For example, if we would have a trading signal on one row and another trading signal on the next row, then the current positions would be = 2).
2. Calculate trades, strategy returns, sessions (as usual)
3. Apply sl and tp

-> We can Actually combine the two above:

Pseudo code:
1. Position column = cumsum() the trading signals
2. if max is not None: np.clip(0, max) else: np.clip(0, np.inf)
3. Calculate trades, strategy returns, sessions (as usual)
4. Loop on the len(df):\
	if position != 0:\
			Apply SL and Take Profit\
	\
	elif trading signal != position column (trading signal == 1 and position column == 0): \
		position current row = trading signal current row\
		position column.ffill()

Note that we want correlated signals to be merged (trend following or mean reversion (for now)) and uncorrelated 

-> Be in a trade as long as a condition is still true (Ex: Stay in a trade as long as Supertrend is green):
1. Create a position column that is 1 if a condition is true, 0 otherwise
2. Apply sessions as usual
3. Apply take profit and stop loss as usual


In [3]:
num_rows = len(test.index)
num_rows #This includes the row for each coin at each index

NameError: name 'test' is not defined

In [ ]:
#Generate a random signals column
test['signals'] = np.random.choice([0, 1], size = num_rows)
test['signals']

date                 coin   
2020-12-31 19:00:00  BTCUSDT    1
                     ETHUSDT    1
2020-12-31 20:00:00  BTCUSDT    1
                     ETHUSDT    1
2020-12-31 21:00:00  BTCUSDT    0
                               ..
2021-01-09 16:00:00  ETHUSDT    1
2021-01-09 17:00:00  BTCUSDT    1
                     ETHUSDT    1
2021-01-09 18:00:00  BTCUSDT    1
                     ETHUSDT    0
Name: signals, Length: 432, dtype: int32

In [ ]:
series = test['signals']
position = np.where(series == 1, 1, 0)
position = pd.Series(position, index = series.index)
position = position.shift(test.index.get_level_values(1).nunique()).fillna(0) 
position

date                 coin   
2020-12-31 19:00:00  BTCUSDT    0.0
                     ETHUSDT    0.0
2020-12-31 20:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
2020-12-31 21:00:00  BTCUSDT    1.0
                               ... 
2021-01-09 16:00:00  ETHUSDT    0.0
2021-01-09 17:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
2021-01-09 18:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
Length: 432, dtype: float64

In [ ]:
test['position'] = position

In [ ]:
test['position']

date                 coin   
2020-12-31 19:00:00  BTCUSDT    0.0
                     ETHUSDT    0.0
2020-12-31 20:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
2020-12-31 21:00:00  BTCUSDT    1.0
                               ... 
2021-01-09 16:00:00  ETHUSDT    0.0
2021-01-09 17:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
2021-01-09 18:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
Name: position, Length: 432, dtype: float64

In [ ]:
class Position():
    def __init__(self, df):
        """
        parametes:
            df: Stacked dataframe
        """
        self.df = df.copy()            
            

    def custom_positon(self, series):
        """
        This will take a series where a condition is met and return a series with the position
        The series can have values of either 1 (when condition is met) or other values (when condition is not met)

        It is basically a copy of the series while keeping only the values of 1.

        Assumes a stacked column (if the series has a multiindex of multicoins)
        returns the position column
        """
        position = np.where(series == 1, 1, 0)
        position = pd.Series(position, index = series.index)
        position = position.shift(test.index.get_level_values(1).nunique()).fillna(0) 
        return position

In [ ]:
pos = Position(test)
test['position'] = pos.custom_positon(test['signals'])

In [ ]:
test['position']

date                 coin   
2020-12-31 19:00:00  BTCUSDT    0.0
                     ETHUSDT    0.0
2020-12-31 20:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
2020-12-31 21:00:00  BTCUSDT    1.0
                               ... 
2021-01-09 16:00:00  ETHUSDT    0.0
2021-01-09 17:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
2021-01-09 18:00:00  BTCUSDT    1.0
                     ETHUSDT    1.0
Name: position, Length: 432, dtype: float64

Now for the position based on the trading signals

In [ ]:
#Generate a random signals column
test['signals_1'] = np.random.choice([0, 1], size = num_rows)
test['signals_2'] = np.random.choice([0, 1], size = num_rows)

In [ ]:
test['signals_1']

date                 coin   
2020-12-31 19:00:00  BTCUSDT    0
                     ETHUSDT    0
2020-12-31 20:00:00  BTCUSDT    0
                     ETHUSDT    1
2020-12-31 21:00:00  BTCUSDT    0
                               ..
2021-01-09 16:00:00  ETHUSDT    0
2021-01-09 17:00:00  BTCUSDT    1
                     ETHUSDT    0
2021-01-09 18:00:00  BTCUSDT    1
                     ETHUSDT    0
Name: signals_1, Length: 432, dtype: int32

In [ ]:
signal_columns = [col for col in test.columns if 'signals' in col]
_max = 1
_min = 0
for coin in test.index.get_level_values(1).unique():
    for i in range(len(signal_columns)):
        test[f'position_{i}', coin ] =  test[signal_columns[i]].cumsum().shift(test.index.get_level_values(1).nunique()).fillna(0)
        test[f'position_{i}'] = np.clip(test[f'position_{i}'], _min, _max)




    

